In [6]:
'''
Downloading the FoodEx database if not already done from their public wiki
'''
import os
import requests
from tqdm import tqdm    

url = 'https://github.com/openefsa/efsa-catalogues/releases/download/12.0/MTX_FULL_12_0.ecf'
f = 'data/MTX_FULL_12_0.ecf'
force = False
if not os.path.exists(f) or force:
    response = requests.get(url, stream=True)
    with open(f, "wb") as handle:
        for data in tqdm(response.iter_content(chunk_size=1024), unit="kB"):
            handle.write(data)
    print("File downloaded succesfully")
else:
    print("Skipping download, file already exists")


3715kB [00:00, 5787.73kB/s]

File downloaded succesfully


In [24]:
'''
Storing FoodEx database in parent-child tabular form if not already done
'''

import os
import gc
import pandas as pd
from bs4 import BeautifulSoup
from zipfile import ZipFile
from bs4.element import Tag, NavigableString

def main():
    f_ecf = 'data/MTX_FULL_12_0.ecf'
    f_xl = 'data/MTX_FULL_12_0.xlsx'
    force = False
    if not os.path.exists(f_ecf):
        raise FileNotFoundError("Please execute the previous step to download the ecf file")
    print("reading ecf file")
    with ZipFile(f_ecf) as zf:
        with zf.open('MTX_FULL_12_0.xml') as f:
            soup = BeautifulSoup(f, 'xml')
    if force:
        os.unlink(f_xl)
    if not os.path.exists(f_xl):
        with pd.ExcelWriter(f_xl, engine='xlsxwriter') as writer:  
            hierarchies = get_hierarchies(soup)
            print(f"writing {len(hierarchies)} hierarchies to excel and pickle")
            hierarchies.to_excel(writer,sheet_name='Hierarchies')
            hierarchies.to_pickle("data/hierarchies.pickle")
            attrs = get_attributes(soup)
            print(f"writing {len(attrs)} attributes to excel and pickle")
            attrs.to_excel(writer,sheet_name='Attributes') 
            attrs.to_pickle("data/attributes.pickle")
            terms = get_terms(soup)
            print(f"writing {len(terms)} terms to excel and pickle")
            terms.to_excel(writer,sheet_name='Terms')   
            terms.to_pickle("data/terms.pickle")
            print("Dataframes saved successfully")
    else:
        print("Skipping wrinting to file since it exists already")

def leafmap(node, excluded_parents = {}, force_set = False):
    res = {}
    for desc in node.descendants:
        
        if type(desc)==Tag and len(desc.contents) == 1 and type(desc.contents[0]) == NavigableString and (desc.parent.name not in excluded_parents):
            key = desc.name
            val = desc.contents[0].text
            if key not in res:
                res[key] = val if not force_set else {val}
            else:
                if type(res[key]) is str:
                    res[key] = {res[key]}
                res[key].add(val)
    return res

def get_hierarchies(soup):
    hierarchies = pd.DataFrame([leafmap(h) for h in soup.find_all("hierarchy")])
    hierarchies["hierarchyOrder"] = hierarchies["hierarchyOrder"].astype(int)
    return hierarchies.sort_values("hierarchyOrder", ignore_index = True)

def get_attributes(soup):
   attrs = pd.DataFrame([leafmap(h) for h in soup.find_all("attribute")])
   attrs["attributeOrder"] = attrs["attributeOrder"].astype(int)
   return attrs.sort_values("attributeOrder", ignore_index = True)

def get_attributes(soup):
   attrs = pd.DataFrame([leafmap(h) for h in soup.find_all("attribute")])
   attrs["attributeOrder"] = attrs["attributeOrder"].astype(int)
   return attrs.sort_values("attributeOrder", ignore_index = True)

def get_terms(soup):
   nodes = soup.find_all("term")
   dicts = []
   for node in nodes:
       term = leafmap(node, {"hierarchyAssignment", "implicitAttribute", "attributeValues"}) 
       #extracting hierarchy assignements
       found_ha = False
       for ia in node.find_all("implicitAttribute"):
         attr = ia.find("attributeCode").text
         values = {value.text for value in ia.find_all("attributeValue")}
         if attr in term:
             raise KeyError("The implicit attributs is going to override an existuing value, this is unexpected")
         term[attr] = values
       for ha in node.find_all("hierarchyAssignment"):
           dicts.append({**term, **leafmap(ha)})
           found_ha = True
       if not found_ha:
           dicts.append(term)
   df = pd.DataFrame(dicts)
   df["order"] = df["order"].astype(int)
   return df.sort_values(["hierarchyCode", "parentCode", "order"], ignore_index = True)

main()
gc.collect()

reading ecf file
writing 36 hierarchies to excel and pickle
writing 47 attributes to excel and pickle
writing 79901 terms to excel and pickle
Dataframes saved successfully


13675740

In [4]:
#Showing terms
import pandas as pd
def showterms():
    df = pd.read_pickle("data/terms.pickle")
    return df.head(10)
showterms()
    

,termCode,termExtendedName,termScopeNote,version,lastUpdate,validFrom,status,termType,allFacets,detailLevel,hierarchyCode,parentCode,order,reportable,implicitFacets,foodexOldCode,EUFeedReg,A01,GEMSCode,LangualCode,matrixCode,A02,ISSCAAP,taxonomicCode,alpha3Code,prodTreat,prodPack,prodMeth,IFNCode,validTo,EuringsCode,termShortName
0,A000C,African millet grain,The term is deprecated. Corn salad as living p...,10.0,2019-01-21T12:51:33,2014-10-30T00:00:00,DEPRECATED,{r},{A000C#F01.A059Q$F02.A066Q$F27.A000C},{E},MTX,A000B,1,true,"{F01.A059Q, F27.A000C}",{A.01.000028},NaN,{Eleusine coracana subsp. Africana},{GC4601},NaN,{P0500040-003},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-21T12:51:33,NaN,NaN
1,A000G,Oat grain,Cereal grains from the plant classified under ...,8.9,2017-07-20T17:46:33,2014-10-30T00:00:00,APPROVED,{r},{A000G#F01.A059V$F02.A066Q$F27.A000G},{E},MTX,A000F,1,true,"{F27.A000G, F01.A059V}",{A.01.000029},NaN,{Avena sativa L.},{GC0647},NaN,{P0500050-000},{Cultivated oat},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A000H,"Oat grain, red",Cereal grains from the plant classified under ...,8.9,2017-07-20T17:46:33,2014-10-30T00:00:00,APPROVED,{r},{A000H#F01.A059X$F02.A066Q$F27.A000H},{E},MTX,A000F,2,true,"{F01.A059X, F27.A000H}",{A.01.000029},NaN,{Avena byzantina K. Koch},{GC4659},NaN,{P0500050},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A001X,Mixture of grains,The group includes any type of Mixture of grai...,8.9,2017-07-20T17:46:33,2014-10-30T00:00:00,APPROVED,{r},{A001X#F01.A058Z$F02.A066Q$F27.A001X},{C},MTX,A000L,1,true,{F27.A001X},{A.01.000039},NaN,NaN,{GC0080},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A0D9Y,Barley and similar-,The group includes any type of grain from Barl...,8.9,2017-07-20T17:46:33,2015-03-16T23:19:07,APPROVED,{r},{A0D9Y#F01.A058Z$F02.A066Q$F27.A0D9Y},{C},MTX,A000L,2,true,{F27.A0D9Y},{A.01.000019},NaN,NaN,NaN,NaN,{P0500010},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A04KH,Buckwheat and other pseudo-cereals and similar-,The group includes any type of grain from Buck...,8.9,2017-07-20T17:46:33,2014-10-30T00:00:00,APPROVED,{r},{A04KH#F01.A058Z$F02.A066Q$F27.A04KH},{C},MTX,A000L,3,true,{F27.A04KH},{A.01.000027},NaN,NaN,NaN,NaN,{P0500020},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A000Y,Common millet and similar-,The group includes any type of grain from Comm...,8.9,2017-07-20T17:46:33,2014-10-30T00:00:00,APPROVED,{r},{A000Y#F01.A058Z$F02.A066Q$F27.A000Y},{C},MTX,A000L,4,true,{F27.A000Y},{A.01.000028},NaN,NaN,{GC0646},NaN,{P0500040},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,A000S,Maize and similar-,The group includes any type of grain from Maiz...,8.9,2017-07-20T17:46:33,2014-10-30T00:00:00,APPROVED,{r},{A000S#F01.A058Z$F02.A066Q$F27.A000S},{C},MTX,A000L,5,true,{F27.A000S},{A.01.000022},NaN,NaN,{GC4613},NaN,{P0500030},{Corn and similar},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,A000F,Oat and similar-,The group includes any type of grain from Oat ...,8.9,2017-07-20T17:46:33,2014-10-30T00:00:00,APPROVED,{r},{A000F#F01.A058Z$F02.A066Q$F27.A000F},{C},MTX,A000L,6,true,{F27.A000F},{A.01.000029},NaN,NaN,{GC0647},NaN,{P0500050},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,A001C,Rice and similar-,The group includes any type of grain from Rice...,8.9,2017-07-20T17:46:33,2014-10-30T00:00:00,APPROVED,{r},{A001C#F01.A058Z$F02.A066Q$F27.A001C},{C},MTX,A000L,7,true,{F27.A001C},{A.01.000030},NaN,NaN,{GC0649},NaN,{P0500060},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#showing attributes
import pandas as pd
def showattrs():
    df = pd.read_pickle("data/attributes.pickle")
    return df
showattrs()

,code,name,label,scopeNote,attributeReportable,attributeVisible,attributeSearchable,attributeOrder,attributeType,attributeCatalogueCode,attributeSingleOrRepeatable,attributeInheritance,attributeUniqueness,attributeTermCodeAlias,version,lastUpdate,validFrom,status,attributeMaxLength
0,F02,part,Part-nature,This facet describes the nature of the food it...,O,true,true,1,catalogue,MTX.part,single,R,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN
1,F01,source,Source,"This facet describes the plant, animal, other ...",O,true,true,2,catalogue,MTX.source,single,R,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN
2,F27,racsource,Source-commodities,This facet describes the RPC from which an ing...,O,true,true,3,catalogue,MTX.racsource,repeatable,D,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN
3,F28,process,Process,This facet allows recording different characte...,O,true,true,4,catalogue,MTX.process,repeatable,D,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN
4,F04,ingred,Ingredient,This facet collects ingredients and/or flavour...,O,true,true,5,catalogue,MTX.ingred,repeatable,D,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN
5,F06,medium,Surrounding-medium,This facet is intended for food packed in any ...,O,true,true,6,catalogue,MTX.medium,repeatable,D,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN
6,F08,sweet,Sweetening-agent,This facet allows providing information on the...,O,true,true,7,catalogue,MTX.sweet,repeatable,D,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN
7,F09,fort,Fortification-agent,This facet allows providing information on the...,O,true,true,8,catalogue,MTX.fort,repeatable,D,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN
8,F10,qual,Qualitative-info,This facet provides some principal claims rela...,O,true,true,9,catalogue,MTX.qual,repeatable,D,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN
9,F17,cookext,Extent-of-cooking,This facet describes the intensity of heat tre...,O,true,true,10,catalogue,MTX.cookext,repeatable,D,false,false,8.10,2017-08-03T10:14:41,2015-03-16T23:19:07,APPROVED,NaN


In [8]:
#showing hierarchies
import pandas as pd
def showhierarchies():
    df = pd.read_pickle("data/hierarchies.pickle")
    return df
showhierarchies()

,code,name,label,scopeNote,hierarchyApplicability,hierarchyOrder,version,lastUpdate,validFrom,status,hierarchyGroup
0,report,ReportingHierarchy,Reporting hierarchy,Reporting hierarchy,base,1,12.0,2021-02-01T10:45:02,2015-03-16T23:19:07,APPROVED,NaN
1,pest,PesticideResiduesHierarchy,Pesticide residues hierarchy,Pesticide residues hierarchy,base,2,8.8,2017-06-26T17:34:25,2015-03-16T23:19:07,APPROVED,NaN
2,biomo,ZoonosesHierarchy,Zoonoses hierarchy,Zoonoses hierarchy,base,3,12.0,2021-02-01T10:45:02,2015-03-16T23:19:07,APPROVED,NaN
3,feed,FeedHierarchy,Feed hierarchy,Feed hierarchy,base,4,10.3,2019-10-03T18:11:30,2015-03-16T23:19:07,APPROVED,NaN
4,expo,ExposureHierarchy,Exposure hierarchy,Exposure hierarchy,base,5,12.0,2021-02-01T10:45:02,2015-03-16T23:19:07,APPROVED,NaN
5,vetdrug,VetDrugResHierarchy,VetDrugRes hierarchy,VetDrugRes hierarchy,base,6,11.2,2020-06-22T13:39:16,2015-03-16T23:19:07,APPROVED,NaN
6,botanic,Botanicals,Botanicals,Botanicals,base,7,11.1,2020-04-01T17:04:40,2015-03-16T23:19:07,APPROVED,NaN
7,part,PartNature,Part-nature,Part-nature,attribute,8,12.0,2021-02-01T10:45:02,2015-03-16T23:19:07,APPROVED,"{SSD2, MTXFACETS}"
8,source,Source,Source,Source,attribute,9,12.0,2021-02-01T10:45:02,2015-03-16T23:19:07,APPROVED,"{SSD2, MTXFACETS}"
9,racsource,SourceCommodities,Source-commodities,Source-commodities,attribute,10,12.0,2021-02-01T10:45:02,2015-03-16T23:19:07,APPROVED,"{SSD2, MTXFACETS}"
